In [1]:
import pickle
with open("circuits/circuit.pickle", 'rb') as handle:
    qc = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: '../circuit.pickle'

In [ ]:
calibrations = qc.calibrations
rzx_cal = calibrations['rzx']
rx_cal = calibrations['rx']

In [ ]:
rx_cal.keys()

In [ ]:
type(list(rx_cal.values())[0])

In [ ]:
custom_rx_sched = list(rx_cal.values())[0]
print(custom_rx_sched)

In [ ]:
from qiskit import QuantumCircuit
import numpy as np

circuit = QuantumCircuit(1, name='custom_rx')
circuit.rx(0,0)
custom_rx = circuit.to_gate()

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import InstructionProperties
from qiskit_ibm_provider import IBMProvider
from qiskit_experiments.library import InterleavedRB

In [ ]:
#Setting the IBM provider
api_file = "../../ibm_API_key"
with open(api_file, "r") as f:
        api_key = f.readline().strip()
provider = IBMProvider(api_key, instance='ibm-q-ncsu/nc-state/quantum-compiler')
backend = provider.get_backend('ibmq_mumbai')

In [ ]:
backend.target.add_instruction(custom_rx, {(0,): InstructionProperties(calibration=custom_rx_sched)})

In [ ]:
lengths = np.arange(1, 200, 30)
num_samples = 10
seed = 1010
qubits = [0] # make sure it is the qubit added in the backend target

int_exp = InterleavedRB(
    custom_rx, qubits, lengths, num_samples=num_samples, seed=seed, backend=backend)

In [ ]:
int_expdata = int_exp.run(backend).block_for_results()
int_results = int_expdata.analysis_results()

In [ ]:
# View result data
display(int_expdata.figure(0))
for result in int_results:
    print(result)

In [ ]:
int_exp.circuits()[1].draw(output='mpl')

In [ ]:
# you can see that the transpiled circuits contain the calibrations
int_exp._transpiled_circuits()[1].draw(output='mpl')